## You have to work on the [University dataset](https://drive.google.com/drive/folders/1Hs3nRtK_F3h8eg59B4-TD1DEua6g8Klv?usp=sharing). It contains three different university rankings:
*  The Times Higher Education World University Ranking, shortly *Times*,
*  the Academic Ranking of World Universities, shortly *Shanghai*,
*  the Center for World University Rankings, shortly *cwur*.

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com).

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
Times = pd.read_csv('timesData.csv', na_values = '-', thousands= ',')
Shanghai = pd.read_csv('shanghaiData.csv')
cwur = pd.read_csv('cwurData.csv')
school_and_country = pd.read_csv('school_and_country_table.csv')

### Preprocessing

In [3]:
cwur['world_rank'] = cwur['world_rank'].astype(str)

Filling NaN with mean

In [4]:
mean = Times.groupby('university_name')['income'].mean().reset_index()

In [5]:
Times.income.fillna(round(mean.income, 1), inplace=True)

**Check country name**

In [6]:
correct_country_list = list(school_and_country['country'].unique())

Print wrong spelled country names

In [7]:
for i in Times['country']:
    if i not in correct_country_list:
        print(i)

Unisted States of America
Unted Kingdom


Replace wrong names in Times dataset

In [8]:
Times = Times.replace(to_replace='Unisted States of America', value='United States of America')
Times = Times.replace(to_replace='Unted Kingdom', value='United Kingdom')

Final check

In [9]:
for i in Times['country']:
    if i not in correct_country_list:
        print(i)

No more wrong names

**Check school name**

In [10]:
correct_university_list = list(school_and_country['school_name'].unique())

In [11]:
for i in Times['university_name']:
    if i not in correct_university_list:
        print(i)

No wrong spelled university names

### 1. For each university, extract from the *times* dataset the most recent and the least recent data, obtaining two separate dataframes

In [12]:
uni_most_rec = Times.loc[Times.groupby('university_name')['year'].idxmax()]
uni_least_rec = Times.loc[Times.groupby('university_name')['year'].idxmin()]

### 2. For each university, compute the improvement in `income` between the least recent and the most recent data points

In [13]:
income_diff = pd.merge(uni_least_rec[['university_name','income']],uni_most_rec[['income','university_name']], 
                       on = 'university_name', 
                       suffixes = ['_least_rec','_most_rec'])

In [14]:
income_diff['improvement'] = income_diff['income_most_rec'] - income_diff['income_least_rec']
income_diff

,university_name,income_least_rec,income_most_rec,improvement
0,AGH University of Science and Technology,NaN,NaN,NaN
1,Aalborg University,36.4,43.7,7.3
2,Aalto University,61.9,61.6,-0.3
3,Aarhus University,61.5,68.3,6.8
4,Aberystwyth University,35.5,31.3,-4.2
...,...,...,...,...
813,École Normale Supérieure,30.7,37.1,6.4
814,École Normale Supérieure de Lyon,26.1,31.7,5.6
815,École Polytechnique,37.5,82.3,44.8
816,École Polytechnique Fédérale de Lausanne,38.0,65.4,27.4


### 3. Find the university with the largest increase computed in the previous point

In [15]:
income_diff.loc[income_diff['improvement'].idxmax()]

university_name     TU Dresden
income_least_rec          31.9
income_most_rec           99.7
improvement               67.8
Name: 428, dtype: object

**TU Dresden** has the largest income increase

### 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for *Aarhus University* the value is 122-73=49). Notice that some rankings are expressed as a range

In [16]:
cwur = cwur.rename(columns = {'institution' : 'university_name'})

In [17]:
rank_t = Times.loc[Times.groupby('university_name')['year'].idxmax()][['university_name','world_rank','year']]

rank_s = Shanghai.loc[Shanghai.groupby('university_name')['year'].idxmax()][['university_name','world_rank','year']]

rank_c = cwur.loc[cwur.groupby('university_name')['year'].idxmax()][['university_name','world_rank','year']]

rank_tsc = pd.concat([rank_t, rank_s,rank_c])

In [18]:
# Check

rank_tsc[rank_tsc['university_name'] == 'Aarhus University']

,university_name,world_rank,year
1908,Aarhus University,=106,2016
4469,Aarhus University,73,2015
1321,Aarhus University,122,2015


In [19]:
def calcola_rank(stringa):
    
    stringa = str(stringa)
    
    occ1 = re.compile(r'=(\d\d?\d?)')
    found1 = occ1.match(stringa)
    
    occ2 = re.compile(r'(\d\d?\d?\d?)-(\d\d?\d?\d?)')
    found2 = occ2.match(stringa)
    
    if found1:
        return int(found1.group(1))
    
    elif found2:
        return ((int(found2.group(2))+int(found2.group(1)))//2)     # divisione intera

    else:
        return int(stringa)
    
rank_tsc['world_rank'] = rank_tsc['world_rank'].apply(calcola_rank)

In [20]:
# Ora calcolo il massimo e il minimo per ogni università, e poi la loro differenza (che è la massima differenza)

diff_rank = rank_tsc.groupby('university_name', as_index = False)['world_rank'].max()
 
diff_rank.rename(columns = {'world_rank': 'max_rank'}, inplace = True)

In [21]:
diff_rank = pd.merge(diff_rank, rank_tsc.groupby('university_name', as_index = False)['world_rank'].min(),
         on = 'university_name') 

diff_rank.rename(columns = {'world_rank': 'min_rank'}, inplace = True)

diff_rank['difference'] = diff_rank['max_rank'] - diff_rank['min_rank']

diff_rank

,university_name,max_rank,min_rank,difference
0,AGH University of Science and Technology,782,700,82
1,Aalborg University,565,225,340
2,Aalto University,450,275,175
3,Aarhus University,122,73,49
4,Aberystwyth University,814,325,489
...,...,...,...,...
1447,École centrale de Lyon,881,881,0
1448,École normale supérieure - Paris,37,37,0
1449,École normale supérieure de Cachan,721,721,0
1450,École normale supérieure de Lyon,471,471,0


### 5. Consider only the most recent data point of the *times* dataset. Compute the number of male and female students for each country.

Split female/male ratio

In [22]:
uni_most_rec[['female','male']] = uni_most_rec['female_male_ratio'].str.split(" : ", expand=True)

In [23]:
uni_most_rec['female'] = round((uni_most_rec['female'].astype(float)*uni_most_rec['num_students'].astype(float))/100)
uni_most_rec['male'] = round((uni_most_rec['male'].astype(float)*uni_most_rec['num_students'].astype(float))/100)

In [24]:
uni_most_rec.groupby('country')[['male','female','num_students']].sum()

,male,female,num_students
country,,,
Argentina,41182.0,67191.0,108373.0
Australia,321640.0,391736.0,743627.0
Austria,66113.0,68364.0,134477.0
Bangladesh,41393.0,21323.0,62716.0
Belarus,9084.0,20219.0,29303.0
...,...,...,...
Uganda,18670.0,18670.0,37340.0
Ukraine,19250.0,17846.0,37096.0
United Arab Emirates,4931.0,9516.0,14447.0


### 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

Compute ratio in each university

In [25]:
uni_most_rec['students_ratio'] = uni_most_rec['female']/uni_most_rec['male']

Infinite value caused by female university

In [26]:
#NaN inf value
uni_most_rec['students_ratio'] = uni_most_rec['students_ratio'].replace(np.inf,np.NaN)

Compute overall average ratio

In [27]:
average_ratio = uni_most_rec['students_ratio'].mean()
average_ratio

1.08110086544082

In [28]:
bave_stud = uni_most_rec.loc[uni_most_rec['students_ratio'] < average_ratio][['university_name','country',
                                                                              'num_students','students_ratio']]

bave_stud

,university_name,country,num_students,students_ratio
2003,Aalborg University,Denmark,17422.0,0.923170
2056,Aalto University,Finland,16099.0,0.470631
2105,Aberystwyth University,United Kingdom,9252.0,0.923093
2406,Ajou University,South Korea,12706.0,0.492541
2408,Alexandria University,Egypt,127431.0,0.851845
...,...,...,...,...
2104,Zhejiang University,China,47508.0,0.694898
1856,École Normale Supérieure,France,2400.0,0.851852
2013,École Normale Supérieure de Lyon,France,2218.0,0.961096
1904,École Polytechnique,France,2429.0,0.219378


### 7. For each country compute the fraction of students that are in one of the universities computed in the previous point.

In [29]:
stud_be_ave = bave_stud.groupby('country')['num_students'].sum() / uni_most_rec.groupby('country')['num_students'].sum()

In [30]:
stud_be_ave = stud_be_ave.to_frame()

stud_be_ave['tot_students_country'] = uni_most_rec.groupby('country')['num_students'].sum()

In [31]:
stud_be_ave

,num_students,tot_students_country
country,,
Argentina,NaN,108373.0
Australia,0.196651,743627.0
Austria,0.453855,134477.0
Bangladesh,1.000000,62716.0
Belarus,NaN,29303.0
...,...,...
Uganda,1.000000,37340.0
Ukraine,0.611548,37096.0
United Arab Emirates,0.361736,14447.0


### 8. Read the file `educational_attainment_supplementary_data.csv`, discarding any row without `country_name` or `series_name`

In [32]:
attainment = pd.read_csv('educational_attainment_supplementary_data.csv')

In [33]:
attainment = attainment.dropna(subset=['country_name','series_name'])
attainment

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.84323,NaN,NaN
79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.68491,NaN,NaN
79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99412,NaN,NaN
79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.77150,NaN,NaN


### 9. From `attainment` build a dataframe with the same data, but with 4 columns: `country_name`, `series_name`, `year`, `value`

In [34]:
# Lista con solo colonne relative agli anni
l = []

for c in attainment.columns:
    l.append(c)
    
l.remove('country_name')
l.remove('series_name')

In [35]:
# Creazione DataFrame vuoto

education = {'country_name': [], 'series_name': [], 'value': [], 'year': []}
education = pd.DataFrame(data=education)

for anno in l:
    
    # Prima costriusco un df ausiliario con solo i valori per quell'anno
    aux = attainment[attainment[anno].notnull()][['country_name','series_name',anno]]
    aux.rename(columns = {anno : 'value'}, inplace = True)
    aux['year'] = anno  # colonna costante con valori pari a quell'anno
    
    education = pd.concat([education, aux])

In [36]:
education.sort_values(by = ['country_name', 'series_name'])

,country_name,series_name,value,year
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33000,1985
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.44000,1990
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.57000,1995
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.75000,2000
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.86000,2005
...,...,...,...,...
79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,13.84323,2012
79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,16.68491,2012
79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,11.99412,2012
79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,5.77150,2012


**Stack function**

In [37]:
new_attainment = attainment.set_index(['country_name','series_name']).stack(dropna = False).reset_index()

Rename columns

In [38]:
new_attainment = new_attainment.rename(columns = {'level_2' : 'year', 0 : 'value'})
new_attainment

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33000
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1986,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1987,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1990,0.44000
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1991,NaN
...,...,...,...,...
2134345,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2010,NaN
2134346,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2011,NaN
2134347,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2012,9.14588
2134348,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2013,NaN


### 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [39]:
rank_tsc.head()

,university_name,world_rank,year
2405,AGH University of Science and Technology,700,2016
2003,Aalborg University,225,2016
2056,Aalto University,275,2016
1908,Aarhus University,106,2016
2105,Aberystwyth University,325,2016


In [40]:
uni_count = rank_tsc.groupby('university_name').count().rename(columns = {'world_rank': 'n_of_rank'}).reset_index()

# Remove year column
uni_count = uni_count.drop(['year'], axis = 1)
uni_count

,university_name,n_of_rank
0,AGH University of Science and Technology,2
1,Aalborg University,3
2,Aalto University,3
3,Aarhus University,3
4,Aberystwyth University,2
...,...,...
1447,École centrale de Lyon,1
1448,École normale supérieure - Paris,1
1449,École normale supérieure de Cachan,1
1450,École normale supérieure de Lyon,1


### 11. In the times ranking, compute the number of times each university appears

In [41]:
uni_count = Times.groupby('university_name', as_index = False)['year'].count().copy()

uni_count.rename(columns = {'year' : 'n_count'}, inplace = True)

uni_count

,university_name,n_count
0,AGH University of Science and Technology,1
1,Aalborg University,5
2,Aalto University,5
3,Aarhus University,6
4,Aberystwyth University,5
...,...,...
813,École Normale Supérieure,6
814,École Normale Supérieure de Lyon,6
815,École Polytechnique,6
816,École Polytechnique Fédérale de Lausanne,6


### 12. Find the universities that appear at most twice in the times ranking.

In [42]:
uni_count[uni_count['n_count']<=2]

,university_name,n_count
0,AGH University of Science and Technology,1
5,Adam Mickiewicz University,1
6,Aix-Marseille University,1
7,Ajou University,1
9,Alexandru Ioan Cuza University,1
...,...,...
806,Yokohama City University,1
807,Yokohama National University,1
810,Yuan Ze University,2
811,Yıldız Technical University,1


### 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In [43]:
same_rank_ts = pd.merge(Times, Shanghai, on = ['university_name','year','world_rank'])
same_rank_tsc = pd.merge(same_rank_ts, cwur, on = ['university_name','year','world_rank']) 
same_rank_tsc[['university_name','year','world_rank']]

,university_name,year,world_rank
0,Stanford University,2013,2


Seconda interpretazione

In [44]:
same_rank_ts = pd.merge(Times, Shanghai, on = ['university_name','world_rank'])
same_rank_tsc = pd.merge(same_rank_ts, cwur, on = ['university_name','world_rank']) 

In [45]:
set(same_rank_tsc['university_name'])

{'Harvard University',
 'Johns Hopkins University',
 'Princeton University',
 'Stanford University',
 'University of Cambridge',
 'University of Chicago',
 'Yale University'}